<a href="https://colab.research.google.com/github/PSYcoms/Automatic-Number-Plate-Recognition/blob/main/Train_YoloV3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Wed Oct 12 14:34:03 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
'01 Knapsack using DP.gslides'
 10.mp4
 11.mp4
 12.mp4
 13.mp4
 14.mp4
 15.jpg
 1639584148407.jpg
'18csc203j ct1 mcq with answers.docx'
'18CSC205J Operating System-Unit 2.gslides'
'18 MAB 204T -QB-merged.pdf'
 1.mp4
'1.Part A.doc'
 1.pdf
'1. Types of Inheritance.gslides'
 2.mp4
'2.Part B.doc'
 3.mp4
'3.Part C.doc'
 4.mp4
 5.mp4
 6.mp4
 7.mp4
 8.mp4
 9.mp4
'Abstract Class.gslides'
'ADE_assignment1_Anshuman(RA1911026010117).pdf'
 ADE-assignment2-Q1.pdf
 Ade-assignment2-RA1911026010117.gdoc
 Ade-assignment2-RA1911026010117.pdf
 ADE_assignment2_RA1911026010117.pdf
'ADE_Ex 7 (1).gdoc'
'ADE_Ex 7.docx'
'ADE_Ex 7.gdoc'
'Ade Exp.10.counter Anshuman(RA1911026010117).docx'
 ADE_Exp11_Anshuman_RA1911026010117.docx
'ADE_Exp 1 (1).docx'
'ADE_Exp 1 (1).gdoc'
'ADE_Exp 1 (2).gdoc'
'ADE_Exp 1.docx'
'ADE_Exp 1.gdoc'
'ADE_Exp 2 By Anshuman.docx'
'ADE_Exp 3_Anshuman_RA191102601

**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15494, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 15494 (delta 0), reused 2 (delta 0), pack-reused 15490
Receiving objects: 100% (15494/15494), 14.11 MiB | 21.00 MiB/s, done.
Resolving deltas: 100% (10409/10409), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
!echo "license_plate" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2022-10-12 14:35:22--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  64.5MB/s    in 2.4s    

2022-10-12 14:35:25 (64.5 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Streaming output truncated to the last 5000 lines.
  inflating: data/obj/monitoring_system_2096.JPG  
  inflating: data/obj/monitoring_system_2096.txt  
  inflating: data/obj/monitoring_system_2097.JPG  
  inflating: data/obj/monitoring_system_2097.txt  
  inflating: data/obj/monitoring_system_2098.JPG  
  inflating: data/obj/monitoring_system_2098.txt  
  inflating: data/obj/monitoring_system_2099.JPG  
  inflating: data/obj/monitoring_system_2099.txt  
  inflating: data/obj/monitoring_system_21.JPG  
  inflating: data/obj/monitoring_system_21.txt  
  inflating: data/obj/monitoring_system_210.JPG  
  inflating: data/obj/monitoring_system_210.txt  
  inflating: data/obj/monitoring_system_2100.JPG  
  inflating: data/obj/monitoring_system_2100.txt  
  inflating: data/obj/monitoring_system_2101.JPG  
  inflating: data/obj/monitoring_system_2101.txt  
  inflating: data/obj/monitoring_system_2102.JPG  
  inflating: data/obj/monitoring_system_2102.txt  
  inflating: data/obj/monitoring_syst

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

Streaming output truncated to the last 5000 lines.
0 0.656875 0.617500 0.131250 0.058333

['0', '0.656875', '0.617500', '0.131250', '0.058333']
3724 data/obj/monitoring_system_219.txt
0 0.656875 0.617500 0.131250 0.058333
0 0.427001 0.510689 0.497801 0.247031

['0', '0.427001', '0.510689', '0.497801', '0.247031']
3725 data/obj/cropped_parking_lot_278.txt
0 0.427001 0.510689 0.497801 0.247031
0 0.660625 0.173333 0.091250 0.043333

['0', '0.660625', '0.173333', '0.091250', '0.043333']
3726 data/obj/monitoring_system_1236.txt
0 0.660625 0.173333 0.091250 0.043333
0 0.290000 0.561667 0.120000 0.056667

['0', '0.290000', '0.561667', '0.120000', '0.056667']
3726 data/obj/monitoring_system_1236.txt
0 0.290000 0.561667 0.120000 0.056667
0 0.614375 0.161667 0.091250 0.043333

['0', '0.614375', '0.161667', '0.091250', '0.043333']
3727 data/obj/monitoring_system_2234.txt
0 0.614375 0.161667 0.091250 0.043333
0 0.752500 0.220000 0.090000 0.046667

['0', '0.752500', '0.220000', '0.090000', '0.04666

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/TN7_jpg.rf.cbce355ee6336bace5cb8570bf8df880.jpg', 'data/obj/HP22_jpg.rf.36265d133d00178c1c64d693ec4874e0.jpg', 'data/obj/KL5_jpg.rf.1b726ef8cc1674fa0165bc3604b8b11d.jpg', 'data/obj/RJ7_jpg.rf.363bdaf68a02cad3bc93aec17775a124.jpg', 'data/obj/GA10_jpg.rf.4f29c15a57a1bf9f4e434a9f60ebd13c.jpg', 'data/obj/UP4_jpg.rf.ae465673e0e92dd6062ff852e179d050.jpg', 'data/obj/frame-260.jpg', 'data/obj/ML11_jpg.rf.d939cec2dd6529ecb5ce593b24f8f53d.jpg', 'data/obj/TS15_jpg.rf.a13f9faf091a682d09e2eefac342b0c0.jpg', 'data/obj/KA24_jpg.rf.d60128fdcafa18ed84a91acf0e80f8fa.jpg', 'data/obj/KL14_jpg.rf.cc3db3c69bfa98c6bc65930d8484727b.jpg', 'data/obj/frame-2010.jpg', 'data/obj/frame-200.jpg', 'data/obj/AP39_jpg.rf.f34ff7c78fdd2ef5c5bb81e107bbca40.jpg', 'data/obj/frame-3440.jpg', 'data/obj/frame-4520.jpg', 'data/obj/DN3_jpg.rf.645551cae51e249b39b37531a61f86b8.jpg', 'data/obj/CG10_jpg.rf.282a1e9059bbb9d4518b6ca16262b129.jpg', 'data/obj/DL4_jpg.rf.677289dda8d5c6374cbcee6a509f3ed9.jpg', 'data/obj/CG20_jpg

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.011906, iou_loss = 0.000000, total_loss = 0.011906 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.747498), count: 2, class_loss = 0.009444, iou_loss = 0.115576, total_loss = 0.125020 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.770526), count: 2, class_loss = 0.036774, iou_loss = 0.061464, total_loss = 0.098237 
 total_bbox = 101206, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.847674), count: 2, class_loss = 0.001326, iou_loss = 0.041411, total_loss = 0.042737 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.0